# q3fer/distilbert-base-fallacy-classification

### Import necessary packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix
import mlflow
from mlflow.transformers import log_model
import logging 
from logging import getLogger
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch


from sklearn.model_selection import train_test_split
#  import pickle
import warnings # why? 
from mlflow.sklearn import save_model 
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # This tells Hugging Face: “Don’t use parallel tokenization — avoid possible deadlocks.”

import seaborn as sns
import matplotlib.pyplot as plt

import config 

/Users/alicepope/Capstone project/Capstone_project/backend/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from basic_functions import (
    get_eval_metrics,
    createTrainer, 
    get_encode_tokenize_data
    )

In [3]:
MODEL_NAME = "q3fer/distilbert-base-fallacy-classification" # pulls the fallacy trained model
TRACKING_URI = config.TRACKING_URI
EXPERIMENT_NAME = config.EXPERIMENT_NAME
configuration = AutoConfig.from_pretrained(MODEL_NAME)

logging.basicConfig(format="%(asctime)s: %(message)s") # Configure logging format to show timestamp before every message

logger = logging.getLogger()
logger.setLevel(logging.INFO) # Only show logs that are INFO or more important (e.g., WARNING, ERROR) — but ignore DEBUG.

In [4]:
# Load model config to inspect label mappings

print("Model Label ID to Label Mapping:")
print(configuration.id2label)

print("\nReverse Mapping (Label to ID):")
print(configuration.label2id)

Model Label ID to Label Mapping:
{0: 'ad hominem', 1: 'ad populum', 2: 'appeal to emotion', 3: 'circular reasoning', 4: 'equivocation', 5: 'fallacy of credibility', 6: 'fallacy of extension', 7: 'fallacy of logic', 8: 'fallacy of relevance', 9: 'false causality', 10: 'false dilemma', 11: 'faulty generalization', 12: 'intentional', 13: 'miscellaneous'}

Reverse Mapping (Label to ID):
{'ad hominem': 0, 'ad populum': 1, 'appeal to emotion': 2, 'circular reasoning': 3, 'equivocation': 4, 'fallacy of credibility': 5, 'fallacy of extension': 6, 'fallacy of logic': 7, 'fallacy of relevance': 8, 'false causality': 9, 'false dilemma': 10, 'faulty generalization': 11, 'intentional': 12, 'miscellaneous': 13}


In [5]:
DATA_PATH = "../data/data_dropped_duplicates_small.csv"
MODEL_PATH = "q3fer/distilbert-base-fallacy-classification"
MODEL_TRAINING_PATH ="q3fer/distilbert-base-fallacy-classification"
OUTPUT_DIR = "../models/q3fer/distilbert-base-fallacy-classification/trainer_output"
SAVE_PATH = "../models/q3fer/distilbert-base-fallacy-classification/pytorch_model"

In [6]:
MODEL_PATH
DATA_PATH

'../data/data_dropped_duplicates_small.csv'

### Get data

In [7]:
train_dataset, test_dataset, y_train, le = get_encode_tokenize_data(DATA_PATH, MODEL_PATH)

INFO:basic_functions:../data/data_dropped_duplicates_small.csv
INFO:basic_functions:q3fer/distilbert-base-fallacy-classification
INFO:basic_functions:Loading data...
INFO:basic_functions:0                  ad_hominem
1                        none
2           appeal_to_emotion
3           appeal_to_emotion
4       faulty_generalization
                ...          
4995        appeal_to_emotion
4996    faulty_generalization
4997               ad_hominem
4998                     none
4999                     none
Name: logical_fallacies, Length: 5000, dtype: object
INFO:basic_functions:Train test split, test-size 0.3
INFO:root:encode the label column
INFO:root:tokenize
INFO:basic_functions:create tokenizer & load model
--- Logging error ---
Traceback (most recent call last):
  File "/Users/alicepope/.pyenv/versions/3.11.3/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/Users/alicepope/.pyenv/versions/3.11.3/lib/

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [ ]:
# # Load and preprocess data
# logger.info("Loading and preprocessing data...")
# df = pd.read_csv(DATA_PATH)
# le = LabelEncoder()
# df["label"] = le.fit_transform(df["logical_fallacies"])
# dataset = Dataset.from_pandas(df[["text", "label"]])
# dataset = dataset.train_test_split(test_size=0.3)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# def tokenize(batch):
#     return tokenizer(batch["text"], padding=True, truncation=True)

# dataset = dataset.map(tokenize, batched=True)

In [ ]:
# # checking the labels
# print(le.classes_)  
# print(le.inverse_transform([0,1,2,3,4,5]))

In [ ]:
# Define new labels
id2label = {
    0: "ad_hominem",
    1: "appeal_to_authority",
    2: "appeal_to_emotion",
    3: "false_dilemma",
    4: "faulty_generalization",
    5: "none"
}

label2id = {v: k for k, v in id2label.items()}

### Model Initialization

In [ ]:
num_classes = len(np.unique(y_train))
num_classes

In [ ]:
# Initialize model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=num_classes,
    ignore_mismatched_sizes= True
    )
model.config.id2label = id2label
model.config.label2id = label2id

In [ ]:
final_layer = model.classifier
# Check the size of the weights
print(final_layer.weight.shape)  # Will be [num_labels, hidden_size] (num_labels x hidden_size)

In [ ]:
# To see a summary of the model
print(model.config)

In [ ]:
params = {
      "learning_rate": 5e-5,
      "weight_decay": 0.01,
      "num_train_epochs": 3,
      "evaluation_strategy": "epoch",
      "class_weight":True,
  }


# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)


mlflow.start_run()
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))

mlflow.set_tag("model_name", MODEL_NAME)
mlflow.log_params(params)

In [ ]:
trainer = createTrainer(
    model= model, 
    train_dataset = train_dataset,
    test_dataset = test_dataset,
    output_dir= OUTPUT_DIR, 
    y_train=y_train, 
    class_weight=True, 
    epochs=3, 
    learning_rate=5e-5, 
    weight_decay = 0.01, 
    train_batch_size=4, 
    eval_batch_size=8 )

### Execute Training

In [ ]:
torch.mps.empty_cache()  # Clears unused GPU memory

In [ ]:
# disable upper limit for memory
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# Allows up to 100% of available memory
torch.mps.set_per_process_memory_fraction(1.0)  

In [ ]:
logger.info('training is running')
trainer.train()

### Evaluation

In [ ]:
# Add prediction and reporting here
output = trainer.predict(dataset["test"])
predictions = np.argmax(output.predictions, axis=1)
y_true = output.label_ids

# Classification report
label_names = le.classes_
print(classification_report(y_true, predictions, target_names=label_names))

# Confusion matrix
cm = confusion_matrix(y_true, predictions)
print("Confusion Matrix:")
print(cm)

# plotting
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_names, yticklabels=label_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
path = "../models/distilbert-base-fallacy-classification"
save_model(sk_model=model, path=path)

### Checking the q3fer

In [ ]:
# Run inference with q3fer model on sample texts
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)

examples = [
    "You have no idea what you're talking about; you've only lived here for six months.", # ad_hominem
    "I read a book by a nutritionist who says all carbs are bad.", # appeal_to_authority
    "Can I have the last piece of cake? You know how much I love it, and it's been a tough day for me.", # appeal_to_emotion
    "If we don't order pizza for dinner, we'll have to eat the week-old spaghetti in the fridge.", # false_dilemma
    "I was in Greece for two week. Greeks are amazing people!", # faulty_generalization
    "We should look into the science that supports this idea." # none
]

predictions = pipeline(examples)

for i, pred in enumerate(predictions):
    print(f"Text {i+1}: {examples[i]}")
    print(f"Prediction: {pred['label']} (Score: {pred['score']:.2f})")
    print("-" * 50)

In [ ]:
label_mapping = {
    'ad hominem': 'ad_hominem',
    'appeal to emotion': 'appeal_to_emotion',
    'false dilemma': 'false_dilemma',
    'faulty generalization': 'faulty_generalization',
    'circular reasoning': 'other',
    'appeal to authority': 'appeal_to_authority',  
    'miscellaneous': 'other',  
    'fallacy of logic': 'other',
    'intentional': 'other',
    'ad populum': 'other',
    'equivocation': 'other',
    'false causality': 'other',
    'fallacy of relevance': 'other',
    'fallacy of credibility': 'other',
    'fallacy of extension': 'other'
}

#  the model's label doesn't include 'none'.Maybe we should/could test it out on our dataset with only the fallacies?

In [ ]:
# testing with the examples
mapped_predictions = []

for i, pred in enumerate(predictions):
    original_label = pred['label']
    mapped_label = label_mapping.get(original_label, 'unmapped')

    print(f"Text {i+1}: {examples[i]}")
    print(f"Original Prediction: {original_label} (Score: {pred['score']:.2f})")
    print(f"Mapped to: {mapped_label}")
    print("-" * 50)

    mapped_predictions.append(mapped_label)
